<a href="https://colab.research.google.com/github/arthur-kt/PPP-Intertemporal-Converter/blob/main/PPP_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPP Interporal Converter

## Parâmetros Iniciais

In [70]:
!pip install fredapi
!pip install pppfy
!pip install pycountry

from fredapi import Fred
from pppfy.converter import Converter
import pycountry
import pandas as pd
from datetime import datetime
import requests

# Inicialize o conversor do PPPfy e configure a chave do FRED
c = Converter()
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')

# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('YE').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

## Country 2023 PPP

In [71]:
# Dados externos de PPP para 2022 e 2023
ppp_dados_externos = {
  'AD': {2023: 0.601311820072143},
  'AE': {2023: 2.31884546890079},
  'AG': {2023: 1.83044748608644},
  'AL': {2023: 39.3625610746},
  'AM': {2023: 148.376614712426},
  'AM': {2023: 148.376614712426},
  'AO': {2023: 209.922137539756},
  'AR': {2023: 138.535504351528},
  'AT': {2023: 0.708583},
  'AU': {2023: 1.391128},
  'AZ': {2023: 0.513547203253494},
  'BA': {2023: 0.66728664015},
  'BB': {2023: 2.34255483499186},
  'BD': {2023: 28.6414527647285},
  'BE': {2023: 0.701943},
  'BF': {2023: 194.43591626971},
  'BG': {2023: 0.73855396687},
  'BH': {2023: 0.171277544238009},
  'BI': {2023: 724.166952175717},
  'BJ': {2023: 204.868568650122},
  'BN': {2023: 0.519406037460836},
  'BO': {2023: 2.38405548204567},
  'BR': {2023: 2.43687587390419},
  'BS': {2023: 0.977354359953442},
  'BW': {2023: 5.08539302521326},
  'BY': {2023: 0.765644162330282},
  'BZ': {2023: 1.12538351296439},
  'CA': {2023: 1.169928},
  'CD': {2023: 949.600728138881},
  'CF': {2023: 238.889655747751},
  'CG': {2023: 219.499486949798},
  'CH': {2023: 0.966273},
  'CI': {2023: 212.455422789034},
  'CL': {2023: 431.415905},
  'CM': {2023: 188.690484605277},
  'CN': {2023: 3.63870415348481},
  'CO': {2023: 1401.241791},
  'CR': {2023: 323.002654},
  'CV': {2023: 48.4973794995055},
  'CY': {2023: 0.562049096539},
  'CZ': {2023: 12.55055},
  'DE': {2023: 0.703527},
  'DJ': {2023: 88.9688656185565},
  'DK': {2023: 6.106244},
  'DM': {2023: 1.37371497415892},
  'DO': {2023: 23.4976010939251},
  'DZ': {2023: 41.966491662277},
  'EC': {2023: 0.411672575202021},
  'EE': {2023: 0.562992},
  'EG': {2023: 4.78817690627537},
  'ES': {2023: 0.572592},
  'ET': {2023: 22.171172211744},
  'FI': {2023: 0.764142},
  'FJ': {2023: 0.877554298905316},
  'FM': {2023: 0.946808218876195},
  'FR': {2023: 0.672356},
  'GA': {2023: 232.714326775648},
  'GB': {2023: 0.667418},
  'GD': {2023: 1.60029951889601},
  'GE': {2023: 0.864621897098242},
  'GH': {2023: 3.30353646845364},
  'GM': {2023: 16.3016525087778},
  'GN': {2023: 3339.63879870911},
  'GQ': {2023: 228.929537109749},
  'GR': {2023: 0.516235},
  'GT': {2023: 3.22796367830501},
  'GT': {2023: 3.22796367830501},
  'GW': {2023: 210.840293617353},
  'GY': {2023: 77.8205942538508},
  'HK': {2023: 5.55294423577267},
  'HN': {2023: 11.0790007804808},
  'HR': {2023: 0.432290262501},
  'HT': {2023: 73.3113538690668},
  'HU': {2023: 170.212279},
  'ID': {2023: 4821.59457441777},
  'IE': {2023: 0.751366},
  'IL': {2023: 3.586914},
  'IN': {2023: 20.2166267502145},
  'IQ': {2023: 519.239819509762},
  'IR': {2023: 91081.211147176},
  'IS': {2023: 140.155004},
  'IT': {2023: 0.604018},
  'JM': {2023: 92.3501422303604},
  'JO': {2023: 0.304468780401446},
  'JP': {2023: 94.677395},
  'JP': {2023: 94.677395},
  'KE': {2023: 43.1224591723171},
  'KG': {2023: 24.3658326983705},
  'KH': {2023: 1370.47995628293},
  'KI': {2023: 0.893225826763546},
  'KM': {2023: 187.337048545402},
  'KN': {2023: 1.82301713051001},
  'KR': {2023: 800.348017},
  'KW': {2023: 0.204475956836398},
  'KZ': {2023: 152.354214889111},
  'LA': {2023: 3936.35447275292},
  'LC': {2023: 1.50211728050232},
  'LK': {2023: 86.7353177891082},
  'LR': {2023: 0.439516464456266},
  'LS': {2023: 5.79823538516503},
  'LT': {2023: 0.483176},
  'LU': {2023: 0.82753},
  'LV': {2023: 0.5045},
  'LY': {2023: 1.7961438286171},
  'MA': {2023: 3.81892783199548},
  'MD': {2023: 6.94888047523769},
  'ME': {2023: 0.355971914047},
  'MG': {2023: 1248.82974959173},
  'MH': {2023: 0.902709949689365},
  'MK': {2023: 18.6516828695},
  'ML': {2023: 199.69700312756},
  'MM': {2023: 422.318062837737},
  'MN': {2023: 1103.340170893},
  'MO': {2023: 4.76146075882428},
  'MR': {2023: 11.3448806661197},
  'MT': {2023: 0.561048150939},
  'MU': {2023: 17.5198752901718},
  'MV': {2023: 7.85656491962535},
  'MW': {2023: 418.309235434928},
  'MX': {2023: 9.65993},
  'MY': {2023: 1.42626807760626},
  'MZ': {2023: 23.4570282308295},
  'NA': {2023: 6.85819521829883},
  'NE': {2023: 206.364660706062},
  'NG': {2023: 165.785284949663},
  'NI': {2023: 11.4631845494296},
  'NI': {2023: 11.4631845494296},
  'NL': {2023: 0.739454},
  'NO': {2023: 8.894996},
  'NP': {2023: 33.4071598443357},
  'NR': {2023: 1.41419412769361},
  'NR': {2023: 1.41419412769361},
  'NZ': {2023: 1.460444},
  'OM': {2023: 0.201639189861095},
  'PA': {2023: 0.470127945537058},
  'PE': {2023: 1.74452671883385},
  'PG': {2023: 2.33960072399207},
  'PG': {2023: 2.33960072399207},
  'PH': {2023: 19.2695632962606},
  'PK': {2023: 56.2744628403118},
  'PL': {2023: 1.879248},
  'PR': {2023: 0.771056552808603},
  'PS': {2023: 0.571905822259302},
  'PT': {2023: 0.517344},
  'PT': {2023: 0.517344},
  'PW': {2023: 0.832744639508332},
  'PY': {2023: 2612.61995392143},
  'RO': {2023: 1.75887412014},
  'RS': {2023: 44.9444593312},
  'RU': {2023: 26.6801077041224},
  'RW': {2023: 345.229437397842},
  'SA': {2023: 1.97040765961378},
  'SB': {2023: 6.08033727731148},
  'SC': {2023: 7.6661137781464},
  'SD': {2023: 514.307134863817},
  'SE': {2023: 8.509281},
  'SG': {2023: 0.804086116592075},
  'SG': {2023: 0.804086116592075},
  'SI': {2023: 0.541355},
  'SK': {2023: 0.506852},
  'SL': {2023: 4.99976356177989},
  'SN': {2023: 219.128224654895},
  'SO': {2023: 10867.5164798451},
  'SR': {2023: 10.6044507958133},
  'ST': {2023: 9.79520128561749},
  'SV': {2023: 0.426094939009153},
  'SV': {2023: 0.426094939009153},
  'SX': {2023: 1.36985036088976},
  'SZ': {2023: 5.96839720517122},
  'TC': {2023: 1.2263883735119},
  'TD': {2023: 221.579449198094},
  'TD': {2023: 221.579449198094},
  'TG': {2023: 194.731218631422},
  'TH': {2023: 10.6559974609307},
  'TJ': {2023: 2.53711704890444},
  'TJ': {2023: 2.53711704890444},
  'TL': {2023: 0.322713344756041},
  'TM': {2023: 1.43591360484589},
  'TM': {2023: 1.43591360484589},
  'TN': {2023: 0.884587105183473},
  'TR': {2023: 6.974967},
  'TT': {2023: 3.91967295016779},
  'TT': {2023: 3.91967295016779},
  'TV': {2023: 1.42727299541079},
  'TZ': {2023: 726.476102762249},
  'UA': {2023: 10.5235219853264},
  'UG': {2023: 1228.430688278},
  'US': {2023: 1},
  'UY': {2023: 25.7177624165063},
  'UY': {2023: 25.7177624165063},
  'UZ': {2023: 3012.05243343799},
  'VC': {2023: 1.39641113345768},
  'VN': {2023: 6805.03072394812},
  'VU': {2023: 120.98165140284},
  'WS': {2023: 1.69170325252631},
  'XK': {2023: 0.365662394514768},
  'ZA': {2023: 7.2801946616266},
  'ZM': {2023: 6.70755666282946},
  'ZW': {2023: 1921.54418925218},
}


## Insira seus dados:

In [79]:
# Dados

## Insira em 'País_Moeda' o o país emissor da moeda utilizada (no caso de uso de moedas diferentes do país de origem), em 'País do Programa' o país de origem do Programa, em 'Ano' o ano da origem dos recursos, em 'Gasto_Local' a quantia dispendida, em 'Programa' o nome do Programa e em 'Descrição' uma breve descrição da função ou destinação dos valores.
## A coluna 'Ano2' serve para os programas que tem custos entre 2 diferentes anos, sendo necessário calcular a média do PPP e da inflação americana. Então, insira no 'Ano2' o último ano do programa, senão preencha com 0.

dados = {
    'País_Moeda': ['US',  #1
             'US',  #2
             'US',  #3
             'US',  #4
             'US',  #5
             'KR',  #6
             'KR',  #7
             'KR',  #8
             'KR',  #9
             'KR',  #10
             'GB',  #11
             'GB',  #12
             'GB',  #13
             'GB',  #14
             'GB',  #15
             'GB',  #16
             'US',  #17
             'GB',  #18
             'GB',  #19
             'GB',  #20
             'GB',  #21
             'CN', #22
             'US', #23
             'CN', #24
             'CN', #25
             'CN', #26
             'CN', #27
             'US', #28
             'CN', #29
             'IN', #30
             'IN', #31
             'IN', #32
             'IN', #33
             'IN', #34
             'ZA', #35
             'ZA', #36
             'ZA', #37
             'MX', #38
             'MX', #39
             'MX', #40
             'MX', #41
             'MX', #42
             'MX', #43
             'MX', #44
             'MX',  #45
             'IN', #46
             'KR', #47
             ],  # Usando códigos ISO2 dos países

        'País do Programa': ['US',  #1
             'US',  #2
             'US',  #3
             'US',  #4
             'US',  #5
             'KR',  #6
             'KR',  #7
             'KR',  #8
             'KR',  #9
             'KR',  #10
             'GB',  #11
             'GB',  #12
             'GB',  #13
             'GB',  #14
             'GB',  #15
             'GB',  #16
             'GB',  #17
             'GB',  #18
             'GB',  #19
             'GB',  #20
             'GB',  #21
             'CN', #22
             'CN', #23
             'CN', #24
             'CN', #25
             'CN', #26
             'CN', #27
             'CN', #28
             'CN', #29
             'IN', #30
             'IN', #31
             'IN', #32
             'IN', #33
             'IN', #34
             'ZA', #35
             'ZA', #36
             'ZA', #37
             'MX', #38
             'MX', #39
             'MX', #40
             'MX', #41
             'MX', #42
             'MX', #43
             'MX', #44
             'MX', #45
             'IN', #46
             'KR', #47
             ],  # Usando códigos ISO2 dos países

    'Ano': [2021,  #1
            2021,  #2
            2021,  #3
            2023,  #4
            2022,  #5
            2023,  #6
            2023,  #7
            2023,  #8
            2023,  #9
            2023,  #10
            2023,  #11
            2023,  #12
            2024,  #13
            2017,  #14
            2017,  #15
            2019,  #16
            2017,  #17
            2019,  #18
            2021,  #19
            2022,  #20
            2022,  #21
            2016, #22
            2019, #23
            2009, #24
            2019, #25
            2009, #26
            2020, #27
            2008, #28
            2021, #29
            2012, #30
            2018, #31
            2023, #32
            2020, #33
            2021, #34
            2016, #35
            2012, #36
            2011, #37
            2008, #38
            2007, #39
            2006, #40
            2009, #41
            2013, #42
            2013, #43
            2021, #44
            2021, #45
            2018 #46
            2024, #47
            ],

   'Ano2': [0,  #1
            0,  #2
            0,  #3
            0,  #4
            0,  #5
            0,  #6
            0,  #7
            0,  #8
            0,  #9
            0,  #10
            0,  #11
            0,  #12
            0,  #13
            0,  #14
            0,  #15
            0,  #16
            0,  #17
            0,  #18
            0,  #19
            0,  #20
            0,  #21
            0, #22
            0, #23
            2016, #24
            0, #25
            0, #26
            0, #27
            2020, #28
            0, #29
            2017, #30
            2020, #31
            0, #32
            0, #33
            0, #34
            0, #35
            0, #36
            2015, #37
            0, #38
            0, #39
            0, #40
            0, #41
            0, #42
            0, #43
            0, #44
            0, #45
            2020 #46
            0, #47
            ],

    'Gasto_Local': [4460000000,  #1
                    2650000000,  #2
                    6500000000,  #3
                    2500000000,  #4
                    2105000000,  #5
                    780100000000,  #6
                    49500000000,  #7
                    1880000000000,  #8
                    60400000000,  #9
                    2100000000000,  #10
                    1100000000,  #11
                    17000000,  #12
                    7600000,  #13
                    5000000000,  #14
                    80000000000,  #15
                    4000000000,  #16
                    3000000000,  #17
                    948100000000,  #18
                    560000000,  #19
                    650000000,  #20
                    520000000,  #21
                    16000000000000, #22
                    291000000000, #23
                    5640000000000, #24
                    1861695000000, #25
                    15, #26
                    74, #27
                    3800000000, #28
                    1147000000, #29
                    1250000000, #30
                    4000000000, #31
                    3000000000, #32
                    69400000000, #33
                    5000000000, #34
                    14426000000, #35
                    1600000000, #36
                    13747152, #37
                    4500000000, #38
                    4500000000, #39
                    4000000000, #40
                    1700000000, #41
                    2138280000, #42
                    11900000, #43
                    89139000000, #44
                    44048700000, #45
                    1440000000, #46
                    55000000000, #47
                    ],  # Gasto em moeda local

    'Programa': ['EOs 14017 and 14001',  #1
                 'EOs 14017 and 14001',  #2
                 'ARPA-H',  #3
                 'ARP A-H',  #4
                 'Executive Order on Advancing Biotechnology and Biomanufacturing Innovation',  #5
                 'Orçamento 2024',  #6
                 'Korean ARPA-H',  #7
                 'Korean ARPA-H',  #8
                 'Boston-Korea',  #9
                 'First National Strategic Technology Nurture Plan',  #10
                 'Medicines Manufacturing Industry Partnership (MMIP)',  #11
                 'Life Sciences Innovative Manufacturing Fund (LSIMF)',  #12
                 'Life Sciences Innovative Manufacturing Fund (LSIMF)',  #13
                 'The Life Sciences Industrial Strategy',  #14
                 'The Life Sciences Industrial Strategy',  #15
                 'The Life Sciences Industrial Strategy',  #16
                 'The Life Sciences Industrial Strategy',  #17
                 'The Life Sciences Industrial Strategy',  #18
                 'UK Innovation Strategy',  #19
                 'Life Sciences 2030 Skills Strategy',  #20
                 'Life Sciences 2030 Skills Strategy',  #21
                 'Healthy China 2030', #22
                 'Healthy China 2030', #23
                 'Healthy China 2030', #24
                 '14º Plano Quinquenal Chinês (2021-2025)', #25
                 '14º Plano Quinquenal Chinês (2021-2025)', #26
                 '14º Plano Quinquenal Chinês (2021-2025)', #27
                 '14º Plano Quinquenal para o Desenvolvimento da Bioeconomia', #28
                 'Relatório de autoavaliação sobre o desempenho geral dos fundos de subsídios regulatórios de medicamentos em 2021', #29
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #30
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #31
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #32
                 'Programas de incentivos vinculados (PLI)', #33
                 'Strengthening of Pharmaceutical Industry Scheme', #34
                 'Investimento em P&D GOV 2016', #35
                 'Industrial Policy Action Plan - Ketlaphela', #36
                 'Industrial Policy Action Plan - Ketlaphela', #37
                 'Estímulos Fiscales', #38
                 'Estímulos Fiscales', #39
                 'Estímulos Fiscales', #40
                 'Programa de Estímulos à Inovação (PEI)', #41
                 'Programa de Estímulos à Inovação (PEI', #42
                 'Programa de Estímulos à Inovação (PEI)', #43
                 'Programa Especial de Ciencia, Tecnología e Innovación', #44
                 'Programa Especial de Ciencia, Tecnología e Innovación',  #45
                 'Programa de assistência para atualização de tecnologia farmacêutica', #46
                 'Korean ARPA-H',  #47
                 ],  # Nomes dos programas

    'Descrição': ['Valores gasto',  #1
                  'Investimentos planejados',  #2
                  '2022-2024',  #3
                  '2024-2025',  #4
                  'Objective to improve and expand domestic biomanufacturing production capacity and processes',  #5
                  'Feito em 2023',  #6
                  'Alocado',  #7
                  'Entre 2024-2033',  #8
                  'Sem descrição',  #9
                  'De 2023 a 2028',  #10
                  'Follow the Green, High-Tech Road',  #11
                  'Primeira fase',  #12
                  'Segunda fase (2024)',  #13
                  '-',  #14
                  'Over the next 10 years',  #15
                  'Atingir esse nível de gasto governamental em P&D em 2027',  #16
                  'Gasto governamental em P&D em 2017',  #17
                  '-',  #18
                  '-',  #19
                  '-',  #20
                  'Entre 2025-2030',  #21
                  'De 2016 a 2030', #22
                  'Government R&D spending on biotech', #23
                  'From 2009 - 2016 to improve its healthcare system', #24
                  'Valores não relacionados ao plano: Despesas governamentais com saúde', #25
                  'Valores não relacionados ao plano: Subsídios governamentais na saúde por pessoa 2009', #26
                  'Valores não relacionados ao plano: Subsídios governamentais na saúde por pessoa 2020', #27
                  'Valores não relacionados ao plano: Investing over the period of 2008–2020 in biotechnology R&D', #28
                  'Fundos de subsídio para supervisão de alimentos e medicamentos alocados pelo governo central em 2021: valor para produtos farmacêuticos', #29
                  'Período de 2012 a 2017', #30
                  'Período de 2018 a 2020 (50% Promoção de Parques de IFAs + 50% assistência para parques de dispositivos médicos)', #31
                  'Orçamento ampliado em 2023 até o ano fiscal de 2025-2026', #32
                  'Incentivo para produção doméstica de materiais de partida, intermediários de síntese e IFAs', #33
                  'Orçamento para o período de 2021-2026', #34
                  'Gastos com P&D em 2016 (Geral, não apenas saúde)', #35
                  'Investimento total projetado em 2012', #36
                  'Financiamento de fato de 2011 a 2015', #37
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #38
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #39
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #40
                  '-', #41
                  'Aprox. 2.138,28 milhões para setor Farmacêutico de 2013-2018', #42
                  '11,9 milhões de pesos para biotecnologia não agrícola (2013-2018)', #43
                  'Meta para 2024: 89139 milhões de pesos para temas prioritários', #44
                  '44048,7 milhões de pesos para pesquisa em temas prioritários',  #45
                  'Período de 2018 a 2020', #46
                  'Research funding over the next 5 years' #47
                  ]  # Descrições dos programas
}

df = pd.DataFrame(dados)

## Obter taxa de conversão do dólar 2024

In [73]:
# Função para obter o código de moeda usando o código de país ISO2
def get_currency_code(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        currency = pycountry.currencies.get(numeric=country.numeric)
        return currency.alpha_3 if currency else None
    except AttributeError:
        print(f"Não foi possível encontrar a moeda para o código de país: {country_code}")
        return None

# Função para buscar a taxa de câmbio média do ano de 2024
def get_exchange_rate_2024(currency_code):
    try:
        url = f"https://api.exchangerate-api.com/v4/latest/{currency_code}?access_key=5f749a0aa7b1501d0bec324f453a7336"

        response = requests.get(url)
        data = response.json()
        end_date = datetime.today().strftime('%Y-%m-%d')
        exchange_rate = data['rates'].get('USD')  # Taxa de câmbio para USD
        return exchange_rate
    except Exception as e:
        print(f"Erro ao obter taxa de câmbio para {currency_code}: {e}")
        return None

## Função Conversão

In [96]:
# Função para converter gasto local em dólar ajustado pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Para o ano 2023, usa os dados de PPP externos
    if row['Ano2'] == 0:
      if row['Ano'] == 2023:
          ppp_adjusted = ppp_dados_externos[row['País_Moeda']][row['Ano']]
          gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted
      elif row['Ano'] == 2024:
          currency_code = get_currency_code(row['País_Moeda'])
          if currency_code:
              exchange_rate = get_exchange_rate_2024(currency_code)
              if exchange_rate:
                  gasto_em_dolar_2024 = row['Gasto_Local'] / exchange_rate
                  return gasto_em_dolar_2024
              else:
                  print(f"Taxa de câmbio não disponível para {row['País_Moeda']}.")
                  return None
      else:
          # Para outros anos, usa a biblioteca pppfy
          ppp_adjusted = c.get_price_mapping(
              source_country=row['País_Moeda'],
              source_price=row['Gasto_Local'],
              destination_country="US",
              year=row['Ano']
          )
          gasto_em_dolar_ppp = ppp_adjusted["ppp_adjusted_local_price"]

      # Ajuste pela inflação dos EUA
      cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    else:
      # Caso exista Ano2, calcula o PPP médio entre Ano1 e Ano2
      ppp_ano1 = c.get_price_mapping(
      source_country=row['País_Moeda'],
      source_price=row['Gasto_Local'],
      destination_country="US",
      year=row['Ano']
      )["ppp_adjusted_local_price"]

      ppp_ano2 = c.get_price_mapping(
      source_country=row['País_Moeda'],
      source_price=row['Gasto_Local'],
      destination_country="US",
      year=row['Ano2']
      )["ppp_adjusted_local_price"]

      ppp_adjusted = (ppp_ano1 + ppp_ano2) / 2  # Média do PPP entre Ano1 e Ano2
      gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted

      # Ajuste pela inflação dos EUA usando CPI de Ano1 e Ano2 (se disponível)

      # Obter o índice CPI de ambos os anos
      cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
      cpi_ano2 = cpi_df_ano.loc[str(row['Ano2'])].values[0]

      # Calcula a média do CPI para ajuste pela inflação
      cpi_media = (cpi_ano1 + cpi_ano2) / 2

      # Ajusta o gasto usando a média do CPI
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_media)

    return float(gasto_ajustado)


## Função Formatação

In [75]:
# Função para formatar valores em milhões, bilhões ou trilhões
def formatar_valor(valor):
    if valor >= 1_000_000_000_000:  # Trilhão
        return f"{valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"{valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"{valor / 1_000_000:.2f} milhões"
    else:
        return f"{valor:.2f}"

## Conversor

In [97]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Convertido_Dolar_2024'] = df.apply(converter_para_dolar_corrente, axis=1)

# Aplicar a formatação para as colunas 'Gasto_Local' e 'Gasto_Ajustado_Dolar 2024'
df['Gasto Local'] = df['Gasto_Local'].apply(formatar_valor)
df['Gasto Convertido US$ 2024'] = df['Gasto_Convertido_Dolar_2024'].apply(formatar_valor)

# Resultado final
df[['País do Programa', 'Programa', 'Descrição', 'Ano', 'Ano2', 'Gasto Local', 'Gasto Convertido US$ 2024']]

<ipython-input-96-9809abf9b5b0>:63: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(gasto_ajustado)


,País do Programa,Programa,Descrição,Ano,Ano2,Gasto Local,Gasto Convertido US$ 2024
0,US,EOs 14017 and 14001,Valores gasto,2021,0,4.46 bilhões,5.15 bilhões
1,US,EOs 14017 and 14001,Investimentos planejados,2021,0,2.65 bilhões,3.06 bilhões
2,US,ARPA-H,2022-2024,2021,0,6.50 bilhões,7.50 bilhões
3,US,ARP A-H,2024-2025,2023,0,2.50 bilhões,2.57 bilhões
4,US,Executive Order on Advancing Biotechnology and...,Objective to improve and expand domestic bioma...,2022,0,2.10 bilhões,2.25 bilhões
5,KR,Orçamento 2024,Feito em 2023,2023,0,780.10 bilhões,1.00 bilhões
6,KR,Korean ARPA-H,Alocado,2023,0,49.50 bilhões,63.48 milhões
7,KR,Korean ARPA-H,Entre 2024-2033,2023,0,1.88 trilhões,2.41 bilhões
8,KR,Boston-Korea,Sem descrição,2023,0,60.40 bilhões,77.46 milhões
9,KR,First National Strategic Technology Nurture Plan,De 2023 a 2028,2023,0,2.10 trilhões,2.69 bilhões
